#### Load data - house 1

In [2]:
import pandas as pd

# Load your dataset
file_path = "/Users/karolinenielsen/Dev/ACIT_4620/groupProject/data/ukdale/house_1/house1_2013_1min.csv"
data = pd.read_csv(file_path, parse_dates=["timestamp"], index_col="timestamp")

# column metadata - categories
data.attrs["categories"] = {
    "kitchen": ["dishwasher", "kettle", "toaster", "microwave"],
    "livingroom": ["tv", "htpc", "amp_livingroom", "subwoofer_livingroom"],
    "lights": [ "kitchen_lights", "livingroom_s_lamp", "kitchen_dt_lamp", "bedroom_ds_lamp", "livingroom_s_lamp2", "livingroom_lamp_tv", "kitchen_lamp2", "utility_lamp", "bedroom_d_lamp", "childs_table_lamp", "childs_ds_lamp", "office_lamp1", "office_lamp2", "office_lamp3" ],
    "other": ["washing_machine"]
}

data


,aggregate,washing_machine,dishwasher,tv,kitchen_lights,htpc,kettle,toaster,microwave,amp_livingroom,...,subwoofer_livingroom,livingroom_lamp_tv,kitchen_lamp2,utilityrm_lamp,bedroom_d_lamp,childs_table_lamp,childs_ds_lamp,office_lamp1,office_lamp2,office_lamp3
timestamp,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,507.800,0.0,1.0,94.000,0.0,69.200,1.0,0.0,1.0,23.200,...,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01 00:01:00,525.100,0.0,1.0,108.100,0.0,69.400,1.0,0.0,1.0,23.100,...,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01 00:02:00,519.500,0.0,1.0,101.556,0.0,69.444,1.0,0.0,1.0,23.111,...,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01 00:03:00,512.667,0.0,1.0,97.900,0.0,69.700,1.0,0.0,1.0,23.000,...,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01 00:04:00,512.300,0.0,1.0,91.900,0.0,69.200,1.0,0.0,1.0,23.200,...,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-31 23:55:00,396.800,0.0,1.0,1.000,0.0,64.667,1.0,1.0,1.0,23.444,...,17.333,13.0,18.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-12-31 23:56:00,394.800,0.0,1.0,1.000,0.0,67.250,1.0,1.0,1.0,23.250,...,17.333,13.0,18.111,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-12-31 23:57:00,393.700,0.0,1.0,1.000,0.0,64.625,1.0,1.0,1.0,23.000,...,17.857,13.0,18.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Define universal fuzzy memebership functions

In [1]:
import numpy as np
import skfuzzy as fuzz

# Define power range (0–4000 W is enough for most UK-DALE appliances)
x_power = np.arange(0, 4001, 1)

# Universal fuzzy sets
mf_off = fuzz.trimf(x_power, [0, 0, 10])         # near 0W
mf_low = fuzz.trimf(x_power, [0, 50, 200])       # small standby or dim usage
mf_med = fuzz.trimf(x_power, [150, 800, 1500])   # mid-range power
mf_high = fuzz.trimf(x_power, [1000, 4000, 4000]) # high consumption


# apply to each appliance col
appliance_cols = [col for col in data.columns if col != "aggregate"]

def fuzzify_absolute(series):
    """

    :param series:
    :return:
    """
    off = fuzz.interp_membership(x_power, mf_off, series)
    low = fuzz.interp_membership(x_power, mf_low, series)
    med = fuzz.interp_membership(x_power, mf_med, series)
    high = fuzz.interp_membership(x_power, mf_high, series)
    return pd.DataFrame({
        f"{series.name}_off": off,
        f"{series.name}_low": low,
        f"{series.name}_medium": med,
        f"{series.name}_high": high
    }, index=series.index)

fuzzy_frames = [fuzzify_absolute(data[col]) for col in appliance_cols]
fuzzy_data = pd.concat(fuzzy_frames, axis=1)

fuzzy_data

ModuleNotFoundError: No module named 'skfuzzy'

In [4]:
# check fuzzy degree distributions
fuzzy_data.describe().T[['mean', 'min', 'max']]

,mean,min,max
washing_machine_off,9.511722e-01,0.0,1.000000
washing_machine_low,1.403473e-02,0.0,1.000000
washing_machine_medium,3.426030e-03,0.0,0.999366
washing_machine_high,2.444012e-03,0.0,0.415444
dishwasher_off,8.984185e-01,0.0,1.000000
...,...,...,...
office_lamp2_high,0.000000e+00,0.0,0.000000
office_lamp3_off,9.712924e-01,0.0,1.000000
office_lamp3_low,5.745246e-03,0.0,0.934813
office_lamp3_medium,6.907856e-07,0.0,0.363077


### Join with electricity prices

In [ ]:
# load electricity prices with timestamp and Norway East region (N01)
el_price_all = pd.read_csv("/Users/karolinenielsen/Dev/ACIT_4620/groupProject/data/spotpriser_clean.csv")
el_price = el_price_all[["timestamp", "NO1"]]

# rename NO1 column
el_price = el_price.rename(columns={"NO1": "el_price"})

# --- 1️⃣ Reset indexes so timestamps become columns ---
fuzzy_data = fuzzy_data.reset_index()
el_price = el_price.reset_index()

# --- 2️⃣ Convert to datetime (if not already) ---
fuzzy_data['timestamp'] = pd.to_datetime(fuzzy_data['timestamp'], errors='coerce')
el_price['timestamp'] = pd.to_datetime(el_price['timestamp'], errors='coerce')

# --- 3️⃣ Create a key that represents only the hour of day ---
fuzzy_data['hour'] = fuzzy_data['timestamp'].dt.floor('h').dt.time
el_price['hour'] = el_price['timestamp'].dt.floor('h').dt.time

# --- 4️⃣ Merge by the hour of day ---
merged = fuzzy_data.merge(el_price[['hour', 'el_price']], on='hour', how='left')

merged
